In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
spy = pd.read_csv('SPY.csv',   parse_dates=[0], index_col=[0])
naesx = pd.read_csv('NAESX.csv',   parse_dates=[0], index_col=[0])
irx = pd.read_csv('^IRX.csv',  parse_dates=[0], index_col=[0])

# Compute Ret od NAESX and IRX

In [3]:
naesx['Ret'] = naesx['Adj Close'].pct_change()

In [4]:
irx['Ret'] = irx['Close'].shift()/1200

# Reversal Strategy(without redemption fees)

In [5]:
Return = pd.DataFrame(index = spy.index)
Return['spy_Ret'] = spy['Adj Close'].pct_change()
Return['naesx_Ret'] = naesx['Adj Close'].pct_change()
Return['irx_Ret'] = irx['Close'].shift()/1200

In [6]:
Return['spy_decision'] = 0
Return['naesx_decision'] = 0
Return['spy_decision'].loc[Return['spy_Ret'] <= Return['naesx_Ret']] = 1
Return['naesx_decision'].loc[Return['spy_Ret'] > Return['naesx_Ret']] = 1

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
Return['portfolio_Ret'] = Return['spy_Ret'] * Return['spy_decision'] + Return['naesx_Ret'] * Return['naesx_decision']
Return.dropna(inplace=True)

In [8]:
Return.head()

,spy_Ret,naesx_Ret,irx_Ret,spy_decision,naesx_decision,portfolio_Ret
Date,,,,,,
2000-02-01,-0.015226,0.165087,0.004608,1,0,-0.015226
2000-03-01,0.094134,-0.074362,0.004700,0,1,-0.074362
2000-04-01,-0.032669,-0.051964,0.004767,0,1,-0.051964
2000-05-01,-0.015722,-0.057799,0.004708,0,1,-0.057799
2000-06-01,0.017286,0.094272,0.004575,1,0,0.017286


In [9]:
Return_adj = pd.DataFrame(index = Return.index)
Return_adj['spy'] = Return['spy_Ret'] - Return['irx_Ret']
Return_adj['portfolio'] = Return['portfolio_Ret'] - Return['irx_Ret']

In [10]:
def Sharpe_cal(ret_data):
    output = ret_data.mean()/ret_data.std() * np.sqrt(12)
    return output

def alpha_beta_cal(data):
    model = smf.ols('portfolio ~ spy', data=data).fit()
    
    alpha = model.params.iloc[0]
    beta = model.params.iloc[1]
    return (alpha, beta)

def max_drawdown(ret_data):
    output = ret_data.cummax().iloc[-1] - ret_data.cummin().iloc[-1]
    return output

def Sortino_cal(ret_data, target_ret=0):
    # Return for Sortino
    ret = ret_data - target_ret
    # Downside Risk
    downside_ret = np.minimum(ret_data, target_ret)
    downside_risk = downside_ret.std()
    # Sortino
    output = ret.mean()/downside_risk * np.sqrt(12)
    return output

In [11]:
Sharpe = Sharpe_cal(Return_adj['portfolio'])
Alpha, Beta = alpha_beta_cal(Return_adj)
Drawdown = max_drawdown(Return_adj['portfolio'])
Sortino = Sortino_cal(Return_adj['portfolio'])

In [12]:
Analyzers = pd.Series([Sharpe, Sortino, Alpha, Beta, Drawdown], index = ['Sharpe', 'Sortino', 'Alpha', 'Beta', 'Drawdown'])
Analyzers

Sharpe     -0.376095
Sortino    -0.530197
Alpha      -0.009643
Beta        1.043257
Drawdown    0.330844
dtype: float64

# Reversal Strategy(without redemption consideration)
#### holding NAESX for three months or more

In [13]:
naesx_decision = list(Return.naesx_decision)
holding_period = 0
final_decision = []

for i in naesx_decision:
    if i == 1:
        final_decision.append(1)
        holding_period += 1
    
    if (i == 0) and (holding_period ==0 or holding_period >= 3):
        final_decision.append(0)
        holding_period = 0
        
    if (i == 0) and (holding_period >0 and holding_period < 3):
        final_decision.append(1)
        holding_period += 1

Return['Decision'] = final_decision

In [14]:
Return['portfolio2_Ret'] = Return['spy_Ret'] * (-1*(Return['Decision']-1)) + Return['naesx_Ret'] * Return['Decision']

In [15]:
Return_adj2 = pd.DataFrame(index = Return.index)
Return_adj2['spy'] = Return['spy_Ret'] - Return['irx_Ret']
Return_adj2['portfolio'] = Return['portfolio2_Ret'] - Return['irx_Ret']

In [16]:
Sharpe2 = Sharpe_cal(Return_adj2['portfolio'])
Alpha2, Beta2 = alpha_beta_cal(Return_adj2)
Drawdown2 = max_drawdown(Return_adj2['portfolio'])
Sortino2 = Sortino_cal(Return_adj2['portfolio'])

In [17]:
Analyzers2 = pd.Series([Sharpe2, Sortino2, Alpha2, Beta2, Drawdown2], index = ['Sharpe', 'Sortino', 'Alpha', 'Beta', 'Drawdown'])
Analyzers2

Sharpe     -0.036795
Sortino    -0.055027
Alpha      -0.005008
Beta        1.041049
Drawdown    0.330844
dtype: float64